This notebook contains some boiler plate code for training and deploying models using AWS Sagemaker

# Training
---

In [3]:
from sagemaker import get_execution_role
import sagemaker
from sagemaker import image_uris
from sagemaker.predictor import csv_serializer

## Training using training jobs

In [20]:
session = sagemaker.Session()

# use `get_execution_role` when training through an AWS managed Notebook instance, otherwise use the ARN
# role = get_execution_role()
role = 'arn:aws:iam::994961437140:role/service-role/AmazonSageMaker-ExecutionRole-20220404T135362'

# specify location of train, val and  test data. note - these need to be in the same region
# as the session e.g. session.boto_region_name == s3 region

train_location = 's3://module-2-lesson-2/train.csv'
val_location = 's3://module-2-lesson-2/validation.csv'


#specify bucket
bucket = 'module-2-lesson-2'
# We use this prefix to help us determine where the output will go. 
prefix = 'trained_model_new'


# We need to get the location of the container. 
container = image_uris.retrieve(framework='blazingtext', region='eu-west-2', version='latest')

# Now that we know which container to use, we can construct the estimator object.
blaze = sagemaker.estimator.Estimator(container, # The image name of the training container
                                    role,      # The IAM role to use (our current role in this case)
                                    instance_count=1, # The number of instances to use for training
                                    instance_type='ml.m5.large', # The type of instance to use for training
                                    output_path=f's3://{bucket}/{prefix}/',
                                                                        # Where to save the output (the model artifacts)
                                    sagemaker_session=session) # The current SageMaker session
             
# These hyperparameters are beyond the scope of this course, but you can research the algoirthm here: 
# https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost.html    
    
blaze.set_hyperparameters(mode='supervised')
                        
s3_input_train = sagemaker.inputs.TrainingInput(s3_data=train_location, content_type='csv')
s3_input_validation = sagemaker.inputs.TrainingInput(s3_data=val_location, content_type='csv')

# The fit method launches the training job. 

blaze.fit({'train': s3_input_train, 'validation': s3_input_validation})


Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: latest.


2022-04-14 14:00:21 Starting - Starting the training job...
2022-04-14 14:00:45 Starting - Preparing the instances for trainingProfilerReport-1649944821: InProgress
......
2022-04-14 14:01:46 Downloading - Downloading input data......
2022-04-14 14:02:45 Training - Training image download completed. Training in progress.Arguments: train
[04/14/2022 14:02:45 WARNING 140308162881344] Loggers have already been setup.
[04/14/2022 14:02:45 WARNING 140308162881344] Loggers have already been setup.
[04/14/2022 14:02:45 INFO 140308162881344] nvidia-smi took: 0.025269031524658203 secs to identify 0 gpus
[04/14/2022 14:02:45 INFO 140308162881344] Running single machine CPU BlazingText training using supervised mode.
Number of CPU sockets found in instance is  1
[04/14/2022 14:02:45 INFO 140308162881344] Processing /opt/ml/input/data/train/train.csv . File size: 40.886911392211914 MB
[04/14/2022 14:02:45 INFO 140308162881344] Processing /opt/ml/input/data/validation/validation.csv . File size: 4.

### Training a model locally 

tbc

# Endpoint Creation / Deployment

In [1]:
from sagemaker import get_execution_role
from sagemaker.model import Model
from sagemaker import image_uris

In [2]:

# get the execution role
role = "arn:aws:iam::994961437140:role/service-role/AmazonSageMaker-ExecutionRole-20220413T170815"

# get the image using the "blazingtext" framework and your region
image_uri = image_uris.retrieve(framework='blazingtext', region='eu-west-2', version='latest')

# get the S3 location of a SageMaker model data
model_data = "s3://module-2-lesson-2/trained_model_new/blazingtext-2022-04-14-14-00-21-213/output/model.tar.gz"


# define a model object
model = Model(image_uri=image_uri,model_data=model_data,role=role)

# deploy the model using a single instance of "ml.m5.large"
predictor = model.deploy(initial_instance_count=1, instance_type="ml.m5.large")

Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: latest.


-----!

# Requesting Inferences from deployed model

In [28]:
from sagemaker.predictor import Predictor
import json

# instantiats Predictor using model URI
predictor = Predictor("blazingtext-2022-04-14-14-18-25-847")


In [29]:
payload = {"instances": ['This is a really useful piece of kit and I would recommedn it to anyone\
                         ']}

In [30]:
predictions = predictor.predict(json.dumps(payload),initial_args={'ContentType': 'application/json'})

In [31]:
predictions

b'[{"label": ["__label__1"], "prob": [0.8450029492378235]}]'

In [3]:
# delete endpoint to stop charges
predictor.delete_endpoint()

AttributeError: 'NoneType' object has no attribute 'delete_endpoint'

# Batch Transform Jobs
---